In [ ]:

# yeni hali 

import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt

import os

import tensorflow as tf

from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten,  LSTM , BatchNormalization


In [ ]:
import yfinance as yf

In [ ]:
period = '3000d'

#tickers = ['AAPL', 'MSFT', 'AMZN', 'GOOG', 'GOOGL', 'TSLA', 'NVDA', 'PYPL', 'ADBE','BTC-USD', 'ETH-USD', 'XRP-USD', 'LTC-USD','BCH-USD', 'BNB-USD', 'LINK-USD', 'ADA-USD', 'XLM-USD', 'SOL-USD', 'TRX-USD']

tickers = ['SOL-USD']


In [ ]:
def indir_ve_df_olustur(tickers, period):
    # Her bir hisse senedi için boş bir sözlük oluşturun
    pariteler = {}

    # Her bir hisse senedi için döngü oluşturun ve verileri indirin
    for ticker in tickers:
        try:
            # Hisse senedi verilerini indirin
            veri = yf.download(ticker, period=period)
            
            # Veriyi sözlüğe ekleyin
            pariteler[ticker] = veri
        except Exception as e:
            print(f"{ticker} için veri indirilirken bir hata oluştu: {str(e)}")

    # Her bir hisse senedi için ayrı bir veri çerçevesi oluşturun
    df_listesi = [veri for veri in pariteler.values()]

    return df_listesi

In [ ]:
#pariteler adında klasör oluştur :

import os 

if not os.path.exists('datas/pariteler'):
    os.makedirs('datas/pariteler')



In [ ]:
datasets = indir_ve_df_olustur(tickers, period)

#pariteler içine kaydet 

for i in range(len(datasets)):
    datasets[i].to_csv(f'datas/pariteler/{tickers[i]}.csv')
    #print(f'{tickers[i]} verisi csv olarak kaydedildi')


In [ ]:
window = 30

def veri_hazirla(df, window):
    # Veri çerçevesinin kopyasını oluşturun
    df = df.copy()
    
    # Veri çerçevesine yeni sütunlar ekleyin
    for i in range(1, window + 1):
        df[f'Önceki_{i}_Açılış'] = df['Open'].shift(i)
        df[f'Önceki_{i}_Yüksek'] = df['High'].shift(i)
        df[f'Önceki_{i}_Düşük'] = df['Low'].shift(i)
        df[f'Önceki_{i}_Kapanış'] = df['Close'].shift(i)
        df[f'Önceki_{i}_Adj'] = df['Adj Close'].shift(i)
        df[f'Önceki_{i}_Hacim'] = df['Volume'].shift(i)
        
    # NaN değerleri bırakın
    df.dropna(inplace=True)
    
    return df

# Her bir veri çerçevesi için döngü oluşturun ve verileri hazırlayın
data_windowed = [veri_hazirla(df, window) for df in datasets]


if not os.path.exists('datas/windowed'):
    os.makedirs('datas/windowed')

# Her bir veri çerçevesi için parite ismiyle birlikte windowed klasörüne kaydedin 
for i in range(len(data_windowed)):
    data_windowed[i].to_csv(f'datas/windowed/{tickers[i]}_windowed.csv')
    print(f'{tickers[i]} verisi windowed olarak kaydedildi')



In [ ]:
#data_windowed

In [ ]:
#her bir veri çerçevesi için X  = features(öncekiler) y = labels olan veriyapısını kuruyoruz (labels  = ['Open','High', 'Low', 'Close', 'Adj Close', 'Volume'])

def X_y_olustur(df, window, labels):
    # Veri çerçevesinin kopyasını oluşturun
    df = df.copy()
    
    # Özellikler ve etiketler için boş listeler oluşturun
    X = []
    y = []
    
    # Her bir satır için döngü oluşturun
    for i in range(len(df) - window):
        # Özellikler için satırı alın
        X_row = df.iloc[i:i + window].values
        
        # Etiketler için satırı alın
        y_row = df[labels].iloc[i + window].values
        
        # Özellikleri ve etiketleri listelere ekleyin
        X.append(X_row)
        y.append(y_row)
        
    # Listeleri numpy dizilerine dönüştürün
    X = np.array(X)
    y = np.array(y)
    
    return X, y

# Her bir veri çerçevesi için döngü oluşturun ve X ve y oluşturun
X_y = [X_y_olustur(df, window, ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']) for df in data_windowed]
#X_y = [X_y_olustur(df, window, ['Close' ,'Volume']) for df in data_windowed]

# Her bir veri çerçevesi için parite ismiyle birlikte numpy klasörüne kaydedin

if not os.path.exists('datas/numpy'):
    os.makedirs('datas/numpy')
for i in range(len(X_y)):
    np.save(f'datas/numpy/{tickers[i]}_X.npy', X_y[i][0])
    np.save(f'datas/numpy/{tickers[i]}_y.npy', X_y[i][1])
    print(f'{tickers[i]} verisi X ve y olarak kaydedildi')
    



In [ ]:
# her bir veri çerçevesi için train test split yapalım time series için doğru yolla yapmak gerekiyor

def train_test_split(X, y, test_size):
        # Test boyutunu hesaplayın

        
        #test_size = int(len(X) * test_size) #yüzdelikli tercih değil

        
        # Eğitim ve test veri kümelerini ayırın
        X_train = X[:-test_size]
        X_test = X[-test_size:]
        y_train = y[:-test_size]
        y_test = y[-test_size:]
        
        return X_train, X_test, y_train, y_test

# Her bir veri çerçevesi için döngü oluşturun ve train ve test veri kümelerini ayırın
test_size = 60
X_train_test = [train_test_split(X_y[i][0], X_y[i][1], 60) for i in range(len(X_y))]

In [ ]:
# her bir veri çerçevesi için normalizasyon ve Scaling yapalım

from sklearn.preprocessing import MinMaxScaler

def normalizasyon_yap(X_train, X_test, y_train, y_test):
    # Normalizasyon için ölçekleyici oluşturun
    scaler_X = MinMaxScaler()
    scaler_y = MinMaxScaler()
    
    # Eğitim veri kümelerini ölçekleyin
    X_train = scaler_X.fit_transform(X_train.reshape(-1, X_train.shape[-1])).reshape(X_train.shape)
    y_train = scaler_y.fit_transform(y_train.reshape(-1, y_train.shape[-1])).reshape(y_train.shape)
    
    # Test veri kümelerini ölçekleyin
    X_test = scaler_X.transform(X_test.reshape(-1, X_test.shape[-1])).reshape(X_test.shape)
    y_test = scaler_y.transform(y_test.reshape(-1, y_test.shape[-1])).reshape(y_test.shape)
    
    return X_train, X_test, y_train, y_test, scaler_X, scaler_y

# Her bir veri çerçevesi için döngü oluşturun ve normalizasyon yapın

X_train_test_normalized = [normalizasyon_yap(X_train_test[i][0], X_train_test[i][1], X_train_test[i][2], X_train_test[i][3]) for i in range(len(X_train_test))]



In [ ]:
tf.test.gpu_device_name()


In [ ]:
# her bir veri çerçevesi için model oluşturup eğitelim

def model_olustur(input_shape, output_shape):
    # Modeli oluşturun
    model = Sequential()
    model.add(LSTM(512, input_shape=input_shape, return_sequences=True))
    model.add(Dropout(0.2))
    model.add(BatchNormalization())

    model.add(LSTM(256, return_sequences=True))
    model.add(Dropout(0.2))
    model.add(BatchNormalization())

    model.add(LSTM(256, return_sequences=True))
    model.add(Dropout(0.2))
    model.add(BatchNormalization())

    model.add(LSTM(128, return_sequences=True))
    model.add(Dropout(0.2))
    model.add(BatchNormalization())

    model.add(LSTM(128, return_sequences=True))
    model.add(Dropout(0.2))
    model.add(BatchNormalization())

    model.add(LSTM(64))
    model.add(Dropout(0.2))
    model.add(BatchNormalization())

    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.2))

    model.add(Dense(output_shape))


    
    return model

In [ ]:
# Her bir veri çerçevesi için döngü oluşturun ve modeli oluşturun ve eğitin

if not os.path.exists('datas/models'):
    os.makedirs('datas/models')

models= []

batch_size = 200

epochs = 100

for i in range(len(X_train_test_normalized)):
    # Giriş ve çıkış şeklini alın
    input_shape = X_train_test_normalized[i][0].shape[1:]
    output_shape = X_train_test_normalized[i][2].shape[1]
    
    # Modeli oluşturun
    model = model_olustur(input_shape, output_shape)
    
    # Modeli derleyin
    model.compile(optimizer='adam', loss='mean_squared_error')
    
    # Modeli eğitin
    model.fit(X_train_test_normalized[i][0], X_train_test_normalized[i][2], batch_size=batch_size, epochs=epochs, validation_data=(X_train_test_normalized[i][1], X_train_test_normalized[i][3]))
    
    # Modeli kaydedin
    model.save(f'datas/models/{tickers[i]}_model.h5')
    
    # Modeli listeye ekleyin
    models.append(model)
    print(f'{tickers[i]} modeli eğitildi ve kaydedildi')



In [ ]:
# her bir veri çerçevesi için tahmin yapalım ve gerçekle karşılaştıralım

# Her bir veri çerçevesi için döngü oluşturun ve tahmin yapın

for i in range(len(X_train_test_normalized)):
    # Modeli ve ölçekleyiciyi alın
    model = models[i]
    scaler_X = X_train_test_normalized[i][4]
    scaler_y = X_train_test_normalized[i][5]
    
    # Tahmin yapın
    tahmin = model.predict(X_train_test_normalized[i][1])
    
    # Tahminleri ölçekten çıkarın
    tahmin = scaler_y.inverse_transform(tahmin)
    
    # Gerçek etiketleri ölçekten çıkarın
    gercek = scaler_y.inverse_transform(X_train_test_normalized[i][3])
    
    # Tahminleri ve gerçek etiketleri çizdirin
    plt.figure(figsize=(14, 7))
    plt.plot(gercek, color='red', label='Gerçek')
    plt.plot(tahmin, color='blue', label='Tahmin')
    plt.title(f'{tickers[i]} Tahmin vs Gerçek')
    plt.xlabel('Zaman')
    plt.ylabel('Fiyat')
    plt.legend()
    plt.show()
    print(f'{tickers[i]} için tahminler ve gerçekler çizdirildi')

In [ ]:
#TODO :  normalizasyon yapılacak : zerobase - minmax scaler  +

#TODO : seq2seq LSTM Yapılcak denenecek

#TODO : Dikkat Mekanizması ile LSTM yapılacak